In [ ]:

PATH = 'VRDL/VRDL_HW1_310554010' #you should key in the path of folder that contain this ipynb file
'''
for example, in my setting, I put this ipynb file folder in 'VRDL/VRDL_HW1_310554010' on My Drive
then PATH will be
'VRDL/VRDL_HW1_310554010'
'''

from google.colab import drive
drive.mount('/content/gdrive/')

!ls gdrive/MyDrive/
!nvidia-smi

import os
os.chdir('./gdrive/MyDrive/')

os.chdir(f'./{PATH}')

Mounted at /content/gdrive/
 data	        DLP_final  'Meet Recordings'   tutorial.ipynb   workshop.pdf
'data mining'   hw3.ipynb   review_paper       VRDL
Tue Oct 19 01:39:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+------------

In [ ]:
import random
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import birdloader
from birdloader import BirdImage as BirdImage
import function as fn

In [ ]:
'''
this procedure will generate an answer.txt in your current folder, 
please back to your current folder, and check answer.txt 
'''
#prepare data
testset = BirdImage('test')
test_loader = DataLoader(dataset = testset,
            batch_size = 16,
            shuffle = False,
            num_workers = 0,
            pin_memory = True)
#load data
model1 = models.resnet152(pretrained=False)
model1.fc = nn.Linear(2048, 200)

model2 = models.resnet152(pretrained=False)
model2.fc = nn.Linear(2048, 200)

model3 = models.resnet152(pretrained=False)
model3.fc = nn.Linear(2048, 200)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cuda:0")
model1.to(device)
model1.eval()
model1.load_state_dict(torch.load('./model-para/ep8 acc1.00 0.65.pt', map_location="cuda:0"))

model2.to(device)
model2.eval()
model2.load_state_dict(torch.load('./model-para/ep30 acc0.99 0.65.pt', map_location="cuda:0"))

model3.to(device)
model3.eval()
model3.load_state_dict(torch.load('./model-para/ep8_finetune.pt', map_location="cuda:0"))


#start evaluating
final = torch.empty(0).to(device)
for x in tqdm(test_loader):
  x = x.to(device).float()
  out1 = model1(x)
  out2 = model2(x)
  out3 = model3(x)
  out = out1 + out2 + out3
  out = torch.max(out, 1).indices.view(-1)
  final = torch.cat([final, out], dim=0)
  
#convert index to bird name
final = fn.ind2class(final)
#write final to answer.txt
fn.class2txt(final)

 